<a href="https://colab.research.google.com/github/dabeyrathna/Project2/blob/master/merged.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
root_path = 'gdrive/My Drive/fovea_one_class'
#!pwd
#!cd gdrive/My\ Drive/maskrcnn
os.chdir(root_path)

In [0]:
!pip install tensorflow==1.6
!pip install keras==2.1.5

     |████████████████████████████████| 45.9MB 91kB/s 
     |████████████████████████████████| 3.1MB 44.9MB/s 
     |████████████████████████████████| 890kB 40.0MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=8438e8f3e883365c348833a1df07b8584bd1ff7037888c4c29a43b5f79665e02
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.5
    Uninstalling bleach-3.1.5:
      Successfully uninstalled bleach-3.1.5
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0
     |███████████████████████

In [0]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Root directory of the project
ROOT_DIR = os.path.abspath("")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import food

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
FOOD_WEIGHTS_PATH = "/path/to/mask_rcnn_balloon.h5"  # TODO: update this path

In [0]:
config = food.FoodConfig()
FOOD_DIR = os.path.join(ROOT_DIR, "fovea")

In [0]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()




Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTA

In [0]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:1"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [0]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [0]:
# Load validation dataset
dataset = food.FoodDataset()

dataset.load_food(FOOD_DIR, "val")
# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

[{'name': 'polygon', 'all_points_x': [271, 264, 261, 261, 263, 270, 279, 284, 285, 285, 282, 278, 271], 'all_points_y': [370, 366, 358, 350, 346, 343, 346, 349, 356, 364, 367, 370, 370]}]
[{'food': '3'}]
[1]
[{'name': 'polygon', 'all_points_x': [239, 242, 250, 261, 268, 273, 271, 266, 257, 243, 239, 239], 'all_points_y': [282, 272, 266, 263, 266, 278, 288, 298, 301, 295, 289, 282]}]
[{'food': '3'}]
[1]
[{'name': 'polygon', 'all_points_x': [303, 298, 290, 286, 285, 285, 293, 301, 310, 316, 318, 319, 317, 311, 303], 'all_points_y': [286, 286, 283, 278, 268, 258, 251, 250, 253, 256, 263, 273, 281, 286, 286]}]
[{'food': '3'}]
[1]
[{'name': 'polygon', 'all_points_x': [209, 202, 197, 195, 198, 203, 210, 217, 222, 226, 226, 222, 217, 209], 'all_points_y': [289, 288, 278, 265, 254, 248, 248, 248, 254, 266, 277, 283, 290, 289]}]
[{'food': '3'}]
[1]
[{'name': 'polygon', 'all_points_x': [171, 164, 157, 155, 158, 168, 180, 190, 195, 196, 194, 186, 180, 171], 'all_points_y': [308, 305, 296, 283, 27

In [0]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [0]:
# Set path to balloon weights file

# Download file from the Releases page and set its path
# https://github.com/matterport/Mask_RCNN/releases
weights_path = "logs/lesion20200530T0305/mask_rcnn_lesion_0017.h5"
# Or, load the last model you trained
#weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  logs/lesion20200530T0305/mask_rcnn_lesion_0017.h5
Re-starting from epoch 17


In [0]:
image_id = random.choice(dataset.image_ids)
image_id_les = random.choice(dataset_les.image_ids)
image_id = 1
image_id_les = 1

image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

info = dataset.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset.image_reference(image_id)))


image_les, image_meta_lesh, gt_class_id_les, gt_bbox_les, gt_mask_les =\
    modellib.load_image_gt(dataset_les, config_les, image_id_les, use_mini_mask=False)
    
info_les = dataset_les.image_info[image_id_les]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id_les, 
                                       dataset_les.image_reference(image_id_les)))


results = model.detect([image], verbose=1)
results_les = model_les.detect([image_les], verbose=1)
# Display results
ax = get_ax(1)
r = results[0]
#r_les = results_les[0]

visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset.class_names, r['scores'], ax=ax,
                            title="Predictions")

# visualize.display_instances(image_les, r_les['rois'], r_les['masks'], r_les['class_ids'], 
#                             dataset_les.class_names, r_les['scores'], ax=ax,
#                             title="Predictions_les")

# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

info['class_ids']= [1]
image ID: food.ARG-212OD_LAB_SP_NS.jpg (1) /content/gdrive/My Drive/fovea_one_class/fovea/val/ARG-212OD_LAB_SP_NS.jpg
info['class_ids']= [2]
image ID: food.ARG-212OD_LAB_SP_NS.jpg (1) /content/gdrive/My Drive/fovea_one_class/fovea/val/ARG-212OD_LAB_SP_NS.jpg


AssertionError: ignored